<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/RAG_app/RAG_app_law_QA_ReAct_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community gradio openai chromadb tiktoken langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import files
import os, json

uploaded = files.upload()
filename = list(uploaded.keys())[0]

# JSON 파일 읽기
with open(filename, 'r') as json_file:
    data = json.load(json_file)

os.environ["OPENAI_API_KEY"] = data.get("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = data.get("TAVILY_API_KEY")

Saving api_keys.json to api_keys.json


In [4]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters.base import Language
from langchain_community.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.chat_models import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import (
    HumanMessage,AIMessage
)
from langchain.tools import tool
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.prompts import MessagesPlaceholder
from langchain.agents import create_react_agent
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.agent import AgentFinish

import gradio as gr
import warnings
import openai

warnings.filterwarnings('ignore')

In [5]:
class ChromaEmbedding:
    def __init__(self, directory, embedding):
        """
        :param directory: 벡터 데이터베이스 폴더
        :param embedding: 임베딩을 수행할 모델
        """
        self.directory = directory
        self.chromaDb = Chroma(persist_directory=self.directory, embedding_function=embedding)

    def addJSONL(self, jsonl_file):

        with open(jsonl_file, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        documents = []
        for line in lines:
            data = json.loads(line)
            question = data.get("question", "")
            answer = data.get("answer", "")
            content = f"Question: {question}\nAnswer: {answer}"
            documents.append(content)

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=900,
            chunk_overlap=0,
            length_function=len,
        )

        # 문자열을 지정된 크기의 청크로 분할한다.
        docs = text_splitter.create_documents(documents)

        # 분할된 청크를 임베딩 데이터베이스에 저장한다.
        self.chromaDb.add_documents(docs)
        self.chromaDb.persist()

# 벡터 데이터베이스 폴더
INDEX = "/content/drive/MyDrive/AI-modeling/law_RAG app/app_QA_index"

def buildIndex():
    chroma = ChromaEmbedding(INDEX, OpenAIEmbeddings())
    chroma.addJSONL("/content/drive/MyDrive/AI-modeling/law_RAG app/data/law_qa_sample.jsonl")
    print("임베딩 완료!")

# retriever
chroma = ChromaEmbedding(INDEX, OpenAIEmbeddings())
retriever = chroma.chromaDb.as_retriever(
    search_type="similarity",
    search_kwargs={
        'k': 2,  # 리턴 문서 수
    }
)

In [6]:
source_text = ""

@tool
def law_qa_retrieval(query_retrieval: str) -> str:
    """Retrieval based on law Q&A and get information."""
    print('law_qa_retrieval start')

    qa_interface = RetrievalQA.from_chain_type(llm=llm,
                                               chain_type="stuff",
                                               retriever=retriever,
                                               chain_type_kwargs={
                                                   "verbose": True,
                                               },
                                               return_source_documents=True)

    qa_result = qa_interface(query_retrieval)

    # history[-1][1] = qa_result["result"]
    global source_text
    source_text = ""

    for ix, doc in enumerate(qa_result['source_documents']):
        source_text += f"## 검색문서 {ix+1}\n```\n{doc.page_content}\n```\n\n"


    return qa_result["result"]

@tool
def react_search(query_search: str) -> str:
    """ReAct search on query and get information."""
    print('react_search start')

    prompt = hub.pull("hwchase17/react")
    react_agent = create_react_agent(llm, [TavilySearchResults(max_results=1)], prompt)
    agent_executor = AgentExecutor(agent=react_agent, tools=[TavilySearchResults(max_results=1)], verbose=True)
    react_result = agent_executor.invoke({"input": query_search})

    global source_text

    source_text = ""

    for item in react_result:

        source_text += item
        source_text += '\n'

    return react_result

In [7]:
# llm
llm=ChatOpenAI(
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model="gpt-3.5-turbo-0125",
    temperature=0
)

# prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            당신은 법률기반 질의응답에 기반해 사용자의 질문에 답변하는 챗봇입니다.
            반드시 전달받은 context에 기반해 답변해야 하며,
            전달받은 context로 답변할 수 없을 시 답변을 절대 하면 안 됩니다.
            Lets' think step by step.
            """
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad") # 메세지 리스트를 전달하는 공간
    ]
)

tools = [react_search, law_qa_retrieval]
functions = [format_tool_to_openai_function(f) for f in tools]
model = llm.bind(functions=functions)

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [8]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input,
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "law_qa_retrieval": law_qa_retrieval,
            "react_search": react_search
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

with gr.Blocks() as rag_react_demo:
      gr.HTML("<h2>법률 사례기반 챗봇</h2>")

      with gr.Row():
          with gr.Column(scale=1):
              chatbot = gr.Chatbot()
              msg = gr.Textbox(value="근로기준법 알려줘.")
          with gr.Column(scale=1):
              gr.HTML("<h3>검색 문서")
              source = gr.Textbox()

      def userHandler(user_message, history):
          return "", history + [[user_message, None]]

      def botHandler(history):
          qa_result = run_agent(history[-1][0]).return_values['output']

          history[-1][1] = qa_result
          global source_text

          return history, source_text

      msg.submit(userHandler, [msg, chatbot], [msg, chatbot], queue=False).then(
        botHandler, chatbot, [chatbot, source]
      )

rag_react_demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://cfd5214c0fef20f892.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


law_qa_retrieval start


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Answer: 「근로기준법」제11조 제1항은 “근로기준법은 상시 5인 이상의 근로자를 사용하는 모든 사업 또는 사업장에 적용한다.”라고 규정하고 있고, 이는 국가·지방자치단체에 의한 사업은 물론 국영기업체, 공익사업체, 사회사업단체, 종교단체가 행하는 모든 사업에 적용됩니다(서울지법 1996. 9. 10. 선고 96가단90373 판결, 대법원 1979. 3. 27. 선고 78다163 판결).이와 같이 「근로기준법」은 이 법의 적용대상이 되는 사업을 한정시키지 않고 모든 사업에 적용되도록 규정하고 있으므로 사업의 개념은 탄력적으로 파악되어야 할 것입니다.여기서 사업 또는 사업장의 판단기준과 관련하여 문제되는 것은 다음과 같습니다.첫째, 한 사업주가 2개 이상의 사업장을 갖는 경우에 장소적으로 분산되어 있다 하여도 동일한 조직적 관련을 가지고 사업의 독립성이 없을 경우에는 하나의 사업장으로 보아야 할 것입니다.둘째, 계절적 사업 또는 일용근로자만을 사용하는 사업장의 경우도 같은 법의 적용을 받아야 합니다(대법원 1987. 4. 14. 선고 87도 153 판결).셋째, 외국인사업에 대해서는 외국인회사가 우리나라에서 한국인 노동자를 고용하여 사용자의 지위를 가지는 한, 자연인이나 법인을 불문하고 근로기준법상의 규정 및 의무를 준수·이행하여야 합니다.넷째, 「근로기준법」도 국내법으로서 국내에서만 적용되며 통치권이 미치